Importing the libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
drive.mount('/content/gdrive')
import matplotlib as mpl
import matplotlib.cm as cm
import plotly.graph_objects as go

from sklearn.feature_extraction.text import CountVectorizer
#from sklearn.feature_extraction import stop_words
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

import string
import re

from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

from sklearn.metrics import accuracy_score
import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn import metrics

from time import time
from collections import Counter
from sklearn import model_selection, svm
from sklearn.model_selection import train_test_split
import re
import string
import nltk
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import spacy

import warnings
warnings.filterwarnings("ignore")


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
df = pd.read_csv(r'/content/gdrive/MyDrive/Colab Notebooks/Train_zsoft.csv')
df

,ID,text,label
0,17428,TRANSPORT SYTEX / OEDA,Experience
1,15362,Appliquer la résine Avancé,Skills
2,37153,2020 - 2021 ¢ Formation en ligne sur le,Education
3,10240,Score: 820 (B2),Skills
4,29713,- Anglais :courant - Créole :courant,Skills
...,...,...,...
30124,16850,du Groupe,Experience
30125,6265,@ Mise en place du marché et démarchage d'arti...,Experience
30126,11284,Baccalauréat général série ES 2019,Education
30127,860,Communication en ligne,Experience


**Text preprocessing**

In [ ]:
data=df.copy()
data.head()

,ID,text,label
0,17428,TRANSPORT SYTEX / OEDA,Experience
1,15362,Appliquer la résine Avancé,Skills
2,37153,2020 - 2021 ¢ Formation en ligne sur le,Education
3,10240,Score: 820 (B2),Skills
4,29713,- Anglais :courant - Créole :courant,Skills


In [ ]:
data['text'] = data['text'].apply(lambda x : x.lower())
!pip install contractions

import re
data['text']=data['text'].apply(lambda x: re.sub('\w*\d\w*','', x))

import contractions
data['text'] = data["text"].apply(lambda x : contractions.fix(x))
data['text'] = [re.sub('\n', '', x) for x in data['text']]

import string
data['text'] = data["text"].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x))
data['text'] = data["text"].apply(lambda x: re.sub(' +',' ',x))

import spacy

# Loading model
nlp = spacy.load('en_core_web_sm',disable=['parser', 'ner'])

# Lemmatization with stopwords removal
data['text']=data['text'].apply(lambda x: ' '.join([token.lemma_ for token in list(nlp(x)) if (token.is_stop==False)]))
data['text'] = [re.sub('httpstco', '', x) for x in data['text']]

text_final = data['text']


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.5/287.5 KB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.5/104.5 KB 13.3 MB/s eta 0:00:00


***Labelazing the data***

In [ ]:
data['label'] = data['label'].replace(['Experience', 'Skills', 'Education'], [0, 1, 2])
labels = {0 : 'Experience', 1 : 'Skills', 2 : 'Education'}

**Train and Test data**

In [ ]:
x_train,x_test, y_train, y_test = train_test_split(data['text'], data['label'], test_size = 0.30, shuffle = True, random_state = 1)

**Tokenisation**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
features = cv.fit_transform(x_train)

**Building our models**

*Linear SVC*

In [ ]:
svc =  LinearSVC(loss='hinge',class_weight='balanced',multi_class='ovr',penalty='l2')
%time svc.fit(features, y_train)

CPU times: user 1.68 s, sys: 7.79 ms, total: 1.69 s
Wall time: 1.7 s


LinearSVC(class_weight='balanced', loss='hinge')

In [ ]:
y_pred_train = svc.predict(features)
y_pred_test = svc.predict(cv.transform(x_test))
print("\nTraining Accuracy score:",accuracy_score(y_train, y_pred_train))
print("Testing Accuracy score:",accuracy_score(y_test, y_pred_test))
print(classification_report(y_test, y_pred_test, target_names=[labels[0], labels[1], labels[2]]))


Training Accuracy score: 0.9013750592697961
Testing Accuracy score: 0.7479809713463879
              precision    recall  f1-score   support

  Experience       0.82      0.79      0.80      4893
      Skills       0.62      0.71      0.66      2300
   Education       0.75      0.69      0.72      1846

    accuracy                           0.75      9039
   macro avg       0.73      0.73      0.73      9039
weighted avg       0.75      0.75      0.75      9039



*Bagging with SVC*

In [ ]:
from sklearn.ensemble import BaggingClassifier

model = BaggingClassifier(base_estimator= svc, n_estimators=200)
model.fit(features, y_train)

BaggingClassifier(base_estimator=LinearSVC(class_weight='balanced',
                                           loss='hinge'),
                  n_estimators=200)

In [ ]:
y_pred_train = model.predict(features)
y_pred_test = model.predict(cv.transform(x_test))
print("\nTraining Accuracy score:",accuracy_score(y_train, y_pred_train))
print("Testing Accuracy score:",accuracy_score(y_test, y_pred_test))


Training Accuracy score: 0.9064959696538644
Testing Accuracy score: 0.7495298152450492


*SGD Classifier*

After doing a gridsearchcv in another device

In [ ]:
from sklearn.linear_model import SGDClassifier
sgd = SGDClassifier(loss='hinge', penalty='l2', alpha=0.0001)
sgd.fit(features, y_train)

SGDClassifier()

In [ ]:
y_pred_train = sgd.predict(features)
y_pred_test = sgd.predict(cv.transform(x_test))
print("\nTraining Accuracy score:",accuracy_score(y_train, y_pred_train))
print("Testing Accuracy score:",accuracy_score(y_test, y_pred_test))


Training Accuracy score: 0.8764817449027975
Testing Accuracy score: 0.762805620090718


*Combining Elements*

In [ ]:
from sklearn.ensemble import VotingClassifier

classifiers = ([('Support Vector Machine', svc),
                ('SGD', sgd)])
vc = VotingClassifier(estimators=classifiers)
vc.fit(features, y_train)

VotingClassifier(estimators=[('Support Vector Machine',
                              LinearSVC(class_weight='balanced', loss='hinge')),
                             ('SGD', SGDClassifier())])

In [ ]:
y_pred_train = vc.predict(features)
y_pred_test = vc.predict(cv.transform(x_test))
print("\nTraining Accuracy score:",accuracy_score(y_train, y_pred_train))
print("Testing Accuracy score:",accuracy_score(y_test, y_pred_test))


Training Accuracy score: 0.8817923186344239
Testing Accuracy score: 0.7622524615554818


*Bagging with SGD*

In [ ]:
from sklearn.ensemble import BaggingClassifier

model = BaggingClassifier(base_estimator= sgd, n_estimators=200)
model.fit(features, y_train)
y_pred_train = model.predict(features)
y_pred_test = model.predict(cv.transform(x_test))
print("\nTraining Accuracy score:",accuracy_score(y_train, y_pred_train))
print("Testing Accuracy score:",accuracy_score(y_test, y_pred_test))


Training Accuracy score: 0.880891417733523
Testing Accuracy score: 0.7657926761809934


*Combining Elements*

In [ ]:
from sklearn.ensemble import VotingClassifier

classifiers = ([('Support Vector Machine', svc),
                ('BAG', model)])
vc = VotingClassifier(estimators=classifiers, voting="hard")
vc.fit(features, y_train)
y_pred_train = vc.predict(features)
y_pred_test = vc.predict(cv.transform(x_test))
print("\nTraining Accuracy score:",accuracy_score(y_train, y_pred_train))
print("Testing Accuracy score:",accuracy_score(y_test, y_pred_test))


Training Accuracy score: 0.8834992887624467
Testing Accuracy score: 0.7654607810598517


*Ridge Classifier*

In [ ]:
from sklearn.linear_model import RidgeClassifier

rc = RidgeClassifier(alpha=3.91, max_iter=100000)
rc.fit(features, y_train)
y_pred_train = rc.predict(features)
y_pred_test = rc.predict(cv.transform(x_test))
print("\nTraining Accuracy score:",accuracy_score(y_train, y_pred_train))
print("Testing Accuracy score:",accuracy_score(y_test, y_pred_test))


Training Accuracy score: 0.8750592697961119
Testing Accuracy score: 0.7670096249585131


**Elements together**

In [ ]:
from sklearn.ensemble import VotingClassifier

classifiers = ([('RidgeClassifier', rc),
                ('SVMSGD', vc)])
vc_2 = VotingClassifier(estimators=classifiers, voting="hard")
vc_2.fit(features, y_train)
y_pred_train = vc_2.predict(features)
y_pred_test = vc_2.predict(cv.transform(x_test))
print("\nTraining Accuracy score:",accuracy_score(y_train, y_pred_train))
print("Testing Accuracy score:",accuracy_score(y_test, y_pred_test))


Training Accuracy score: 0.8740161213845424
Testing Accuracy score: 0.7677840469078437


**Results**

In [ ]:
Test_data = pd.read_csv(r'/content/gdrive/MyDrive/Colab Notebooks/Test_zsoft.csv')
Test_data['text'] = Test_data['text'].apply(lambda x : x.lower())
import re
Test_data['text']=Test_data['text'].apply(lambda x: re.sub('\w*\d\w*','', x))
import contractions
Test_data['text'] = Test_data["text"].apply(lambda x : contractions.fix(x))
Test_data['text'] = [re.sub('\n', '', x) for x in Test_data['text']]
import string
Test_data['text'] = Test_data["text"].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x))
Test_data['text'] = Test_data["text"].apply(lambda x: re.sub(' +',' ',x))
import spacy

# Loading model
nlp = spacy.load('en_core_web_sm',disable=['parser', 'ner'])

# Lemmatization with stopwords removal
Test_data['text']=Test_data['text'].apply(lambda x: ' '.join([token.lemma_ for token in list(nlp(x)) if (token.is_stop==False)]))
Test_data['text'] = [re.sub('httpstco', '', x) for x in Test_data['text']]

In [ ]:
fet = cv.transform(Test_data['text'])
y_pred = vc_2.predict(fet)

In [ ]:
predictions = pd.Series(y_pred, name='label')
predictions = predictions.replace([0, 1, 2],['Experience', 'Skills', 'Education'])
predictions

0       Experience
1       Experience
2       Experience
3           Skills
4       Experience
           ...    
7528    Experience
7529    Experience
7530     Education
7531    Experience
7532    Experience
Name: label, Length: 7533, dtype: object

In [ ]:
results = pd.concat([Test_data, predictions],axis=1)
results=results.drop(['text'], axis=1)

In [ ]:
results.to_csv("subs_1.csv",index=False)